In [1]:
import pandas as pd
col_names = ['word', 'pos']
data = pd.read_csv('./Train_utf16.ner', sep=' ', header=None, encoding='utf-16', names=col_names, \
                   skip_blank_lines=False).fillna('')
data.head(30)

,word,pos
0,海,N
1,內,N
2,外,N
3,關,N
4,注,N
5,的,N
6,一,N
7,九,N
8,九,N
9,七,N


In [2]:
data['pos'] = data.apply(lambda row: 'O' if row['pos'] == 'N' else row['pos'], axis=1)
data.head(30)

,word,pos
0,海,O
1,內,O
2,外,O
3,關,O
4,注,O
5,的,O
6,一,O
7,九,O
8,九,O
9,七,O


In [15]:
data.iloc[0].name

0

In [47]:
data.tail(5)

,word,pos
1808531,謀,O
1808532,殺,O
1808533,案,O
1808534,。,O
1808535,,


In [4]:
import numpy as np
y = np.zeros(len(data))
# train, test, y_train, y_test = train_test_split(data, y, test_size=0.3, random_state=42)
limit = int(len(y) * 0.7)
line = -1
def findSplitPoint(row):
    global line
    if line == -1 and int(row.name) >= limit - 1 and row['word'] == '':
        line = int(row.name)
data.apply(findSplitPoint, axis=1)
print (line)

1266021


In [5]:
train = data[0:line+1]
test = data[line+1:]

In [22]:
print (len(train), len(test), len(data))

1266022 542514 1808536


In [24]:
print ((test['pos'] == '').sum())

10920


In [6]:
train.tail(5)

,word,pos
1266017,線,O
1266018,駛,O
1266019,經,O
1266020,。,O
1266021,,


In [7]:
test.head(5)

,word,pos
1266022,前,O
1266023,往,O
1266024,新,B-LOC
1266025,界,I-LOC
1266026,如,O


In [8]:
train.to_csv('./train.data', header=None, index=False, sep=' ', encoding='utf-8')

In [9]:
test.to_csv('./test.data', header=None, index=False, sep=' ', encoding='utf-8')

In [10]:
# pred.data
import pandas as pd
col_names = ['word', 'pos']
data_pred = pd.read_csv('./Test_utf16.ner', sep=' ', header=None, encoding='utf-16', names=col_names, \
                   skip_blank_lines=False).fillna('')

In [11]:
data_pred['pos'] = data_pred.apply(lambda row: '' if row['word'] == '' else 'O', axis=1)
data_pred.head(20)

,word,pos
0,這,O
1,是,O
2,一,O
3,次,O
4,意,O
5,義,O
6,重,O
7,大,O
8,的,O
9,宏,O


In [12]:
data_pred.to_csv('./pred.data', header=None, index=False, sep=' ', encoding='utf-8')

In [20]:
col_names = ['word', 'pos', 'pred']
test_result = pd.read_csv('./test4.rst', sep='\t', header=None, encoding='utf-8', names=col_names, \
                   skip_blank_lines=False).fillna('')
test_result.head(30)

,word,pos,pred
0,前,O,O
1,往,O,O
2,新,B-LOC,B-LOC
3,界,I-LOC,I-LOC
4,如,O,O
5,屯,B-LOC,B-LOC
6,門,I-LOC,I-LOC
7,或,O,O
8,青,B-LOC,B-LOC
9,衣,I-LOC,I-LOC


In [21]:
test_result.to_csv('./test4.result', header=None, index=False, sep=' ', encoding='utf-8')

In [ ]:
# train
# crf_learn -f 4  -p 8 -c 3 template1 train.data model > train.rst
# crf_learn -f 4  -p 8 -c 5 template3 train.data model3 > train3.rst
# crf_learn -f 4  -p 8 -c 5 template3 train.data model4 > train4.rst
# test
# crf_test -m model test.data > test.rst

In [28]:
# eval
col_names = ['word', 'gold', 'pred']
test_rst = pd.read_csv('./test.rst', header=None, sep='\s+', encoding='utf-8', names=col_names)
test_rst.head(5)

,word,gold,pred
0,彈,N,N
1,的,N,N
2,爆,N,N
3,炸,N,N
4,碎,N,N


In [36]:
# precision, recall, f1
rows = test_rst.apply(lambda row: row['gold'] == row['pred'] and row['pred'] != 'N', axis=1)
gold_nums = test_rst['gold'] != 'N' 
pred_nums = test_rst['pred'] != 'N'
p = 1.0 * rows.sum() / pred_nums.sum()
r = 1.0 * rows.sum() / gold_nums.sum()
print 'Precision:', p
print 'Recall:', r
print 'F1', 2*p*r / (p+r)

Precision: 0.955028911493
Recall: 0.885633769741
F1 0.919023201503


In [37]:
# eval model2 c3
col_names = ['word', 'gold', 'pred']
test_rst = pd.read_csv('./test2.rst', header=None, sep='\s+', encoding='utf-8', names=col_names)

rows = test_rst.apply(lambda row: row['gold'] == row['pred'] and row['pred'] != 'N', axis=1)
gold_nums = test_rst['gold'] != 'N' 
pred_nums = test_rst['pred'] != 'N'
p = 1.0 * rows.sum() / pred_nums.sum()
r = 1.0 * rows.sum() / gold_nums.sum()
print 'Precision:', p
print 'Recall:', r
print 'F1', 2*p*r / (p+r)

Precision: 0.962363647962
Recall: 0.885712049159
F1 0.922448230882


In [40]:
# eval model2_c1
col_names = ['word', 'gold', 'pred']
test_rst = pd.read_csv('./test2_c1.rst', header=None, sep='\s+', encoding='utf-8', names=col_names)

rows = test_rst.apply(lambda row: row['gold'] == row['pred'] and row['pred'] != 'N', axis=1)
gold_nums = test_rst['gold'] != 'N' 
pred_nums = test_rst['pred'] != 'N'
p = 1.0 * rows.sum() / pred_nums.sum()
r = 1.0 * rows.sum() / gold_nums.sum()
print 'Precision:', p
print 'Recall:', r
print 'F1', 2*p*r / (p+r)

Precision: 0.963877673739
Recall: 0.878334213977
F1 0.919119830437


In [42]:
# eval model2_c5
col_names = ['word', 'gold', 'pred']
test_rst = pd.read_csv('./test2_c5.rst', header=None, sep='\s+', encoding='utf-8', names=col_names)

rows = test_rst.apply(lambda row: row['gold'] == row['pred'] and row['pred'] != 'N', axis=1)
gold_nums = test_rst['gold'] != 'N' 
pred_nums = test_rst['pred'] != 'N'
p = 1.0 * rows.sum() / pred_nums.sum()
r = 1.0 * rows.sum() / gold_nums.sum()
print 'Precision:', p
print 'Recall:', r
print 'F1', 2*p*r / (p+r)

Precision: 0.958717916298
Recall: 0.890330534844
F1 0.923259565919


In [43]:
# eval model3_c5
col_names = ['word', 'gold', 'pred']
test_rst = pd.read_csv('./test3.rst', header=None, sep='\s+', encoding='utf-8', names=col_names)

rows = test_rst.apply(lambda row: row['gold'] == row['pred'] and row['pred'] != 'N', axis=1)
gold_nums = test_rst['gold'] != 'N' 
pred_nums = test_rst['pred'] != 'N'
p = 1.0 * rows.sum() / pred_nums.sum()
r = 1.0 * rows.sum() / gold_nums.sum()
print 'Precision:', p
print 'Recall:', r
print 'F1', 2*p*r / (p+r)

Precision: 0.958357006081
Recall: 0.891289457719
F1 0.923607308714
